In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import skimage as ski
import scipy.sparse
import time

In [ ]:
def forward_differences(M,N):
    row = np.arange(0,M*N)
    dat = np.ones(M*N)
    col = np.arange(0,M*N).reshape(M,N)
    col_xp = np.hstack([col[:,1:], col[:,-1:]])
    col_yp = np.vstack([col[1:,:], col[-1:,:]])
    
    FD1 = scipy.sparse.coo_matrix((dat, (row, col_xp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))

    FD2 = scipy.sparse.coo_matrix((dat, (row, col_yp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))
    
    FD = scipy.sparse.vstack([FD1, FD2])
    
    return FD

In [ ]:
def prox_l1(u, f, tau):
    return f + np.maximum(0.0, np.abs(u-f)-tau)*np.sign(u-f)

def proj_inf_l2(p, tau):
    # size must be (K,N), l2 over K, inf over N
    norm_p = np.sqrt(np.sum(p**2, axis=0, keepdims=True))
    p /= np.maximum(1, norm_p/tau)
    return p

def proj_inf(p, tau):
    return np.clip(p, -tau, tau)

In [ ]:
# primal dual for TV-l1
def tv_l1_pd(d, lamb=1.0, maxit=1000, check=100, verbose=0):
    
    M,N = d.shape
    u = d.copy()    
    
    d = d.reshape(M*N)
    u = u.reshape(M*N)
    
    # dual variable
    p = np.zeros(M*N*2)

    # make nabla operator
    D = forward_differences(M,N)

    # primal and dual step size
    # tau * sigma * L^2 = 1
    L = np.sqrt(8)
    tau = 1/L
    sigma = 1/tau/L**2
    theta = 1.0
    
    t0 = time.time()
    E = []
    for it in range(0,maxit):

        # remeber old
        u_ = u.copy()

        # primal update        
        u -= tau*(D.T@p)
        
        # proximal step
        u = prox_l1(u,d,tau)
    
        # overrelaxation
        u_ = u + theta*(u-u_)
        
        # dual update
        p += sigma*(D@u_)

        # projection
        p = p.reshape(2,M*N)
        p = proj_inf_l2(p, lamb)
        p = p.reshape(2*M*N)
        
        TV1 = lamb*np.sum(np.sqrt(np.sum(((D@u).reshape(2,M*N))**2, axis=0)))
        energy = TV1 + np.sum(np.abs(u-d))
        E.append(energy)
        if verbose > 0:
            if it%check == check-1:
                

                print("iter = ", it,
                      ", tau = ", "{:.3f}".format(tau),
                      ", sigma = ", "{:.3f}".format(sigma),
                      ", time = ", "{:.3f}".format(time.time()-t0),
                      ", E = ", "{:.6f}".format(energy),
                      end="\r")
                
    return u.reshape(M,N), p.reshape(2,M,N), np.array(E)

In [ ]:
# Load image
g = ski.io.imread("watercastle.png")/255.0
M,N = g.shape

p_l1 = 0.2
f = g.copy()
n = np.random.rand(M,N)
f[n <= p_l1/2] = 0.0
f[n >= (1-p_l1/2)] = 1.0

In [ ]:
# Solve the dual ROF model using accelerated primal-dual
lamb_tv = 0.75
u, p, E = tv_l1_pd(f, maxit=1000, check=100, lamb=lamb_tv, verbose=1)

In [ ]:
plt.figure()
plt.subplot(121)
plt.imshow(f, cmap="gray")

plt.subplot(122)
plt.imshow(u, cmap="gray")